In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from openai import AsyncOpenAI
# import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import requests
import resend

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is no

In [2]:
load_dotenv(override=True)
RESEND_API_KEY = os.getenv("RESEND_API_KEY")

##Agent Workflow

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [9]:
BASE_URL = "https://api.groq.com/openai/v1"
MODEL_NAME='deepseek-r1-distill-llama-70b'

In [10]:
groq_client = AsyncOpenAI(base_url=BASE_URL, api_key=os.getenv("GROQ_API_KEY"))

custom_model = OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=groq_client)

In [11]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=custom_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=custom_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=custom_model
)

In [12]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

<think>
Alright, I need to write a cold sales email for ComplAI, a company that offers a SaaS tool for SOC2 compliance and audit preparation using AI. Let me break this down step by step.

First, I should understand the target audience. SOC2 compliance is typically relevant for businesses that handle sensitive customer data, like SaaS companies, fintech, healthcare, etc. The decision-makers are likely CTOs, CFOs, compliance officers, or auditors. So, the email should address their pain points and offer a solution.

The subject line needs to grab attention. It should mention SOC2 and audits to immediately relate to the recipient's concerns. Maybe something like "Simplify SOC2 Compliance and Streamline Your Next Audit" – that sounds professional and addresses a specific need.

In the opening, I should introduce ComplAI and quickly explain what it does. The goal is to show that we understand their challenges with SOC2 compliance. I should mention the time-consuming and complex nature of a

In [13]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


<think>
Okay, I need to write a cold sales email for ComplAI, a company that offers a SaaS tool for SOC2 compliance and audit preparation, powered by AI. The user wants the email to be professional and serious.

First, I'll start with a strong subject line that grabs attention and clearly states the value proposition. Maybe something like "Simplify SOC2 Compliance with AI-Powered Expertise."

In the greeting, I'll address the recipient by name to make it personal. Then, I'll introduce ComplAI and explain what we do in a concise manner, emphasizing that it's a SaaS tool powered by AI. I should mention the pain points we solve, like the complexity and time-consuming nature of SOC2 compliance.

Next, I'll outline the benefits of using ComplAI. It's important to highlight how our tool makes the process easier, saves time, and reduces stress. Maybe include bullet points for clarity, listing key features like automated controls, real-time monitoring, and audit-ready reports.

I should also a

In [14]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=custom_model
)

In [15]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
<think>
Alright, I need to write a cold sales email for ComplAI, a SaaS tool that helps with SOC2 compliance and audit preparation using AI. The user provided an example email, so I should follow a similar structure but make sure it's original and tailored.

First, I'll start with the subject line. It should be attention-grabbing and relevant. The example used "Simplify SOC2 Compliance with ComplAI" which is good. Maybe I can think of something similar but not identical. Maybe "Streamline Your SOC2 Compliance Journey with ComplAI" or "Effortlessly Achieve SOC2 Compliance with ComplAI."

Next, the greeting. The example used "Dear [Recipient's First Name]," which is standard. I should keep it the same.

In the opening paragraph, the example starts by acknowledging the recipient's role and the importance of SOC2 compliance. I should do the same but perhaps vary the wording a bit. Maybe mention the challenges of keeping up with regulations or the time-consuming nature of 

## Use of Tools

In [16]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=custom_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=custom_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=custom_model
)

In [17]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x107255430>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Tools and Agent integration

In [18]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects via Resend """
    
    # Set up email sender, recipient, and content
    from_email = "shardulggore@gmail.com"  # Replace with your verified sender
    to_email = "shardulgg972001@gmail.com"  # Replace with recipient's email
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"onboarding@resend.dev",
        "to": "shardulggore@gmail.com",
        "subject": "Sales email",
        "html": f"<p>{body}</p>"  # Body wrapped in <p> tags for HTML format
    }
    
    # Send email using Resend API
    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 202:
        return {"status": "success"}
    else:
        return {"status": "failure", "message": response.text}

In [19]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects via Resend', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x106e6aca0>, strict_json_schema=True)

### Convert agent to tool

In [20]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x107e589a0>, strict_json_schema=True)

In [21]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10566fa60>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10566c860>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent

In [22]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model=custom_model)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [23]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=custom_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=custom_model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [24]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects using Resend"""
    
    # Replace with your actual verified sender and target recipient
    from_email = "ed@edwarddonner.com"
    to_email = "shardulggore@gmail.com"
    
    # Get the Resend API key from environment variable
    RESEND_API_KEY = os.environ.get("RESEND_API_KEY")
    
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"Ed Donner <{from_email}>",
        "to": [to_email],
        "subject": subject,
        "html": html_body
    }
    
    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
    
    if response.status_code == 202:
        return {"status": "success"}
    else:
        return {"status": "failure", "message": response.text}


In [25]:
tools = [subject_tool, html_tool, send_html_email]

In [26]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1055320c0>, strict_json_schema=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x107f18900>, strict_json_schema=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects using Resend', params_json_s

In [27]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=custom_model,
    handoff_description="Convert an email to HTML and send it")


In [28]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10566fa60>, strict_json_schema=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10566c860>, strict_json_schema=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent3_

In [29]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=custom_model)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)